In [1]:
import pystan
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("../../RStanBook/chap04/input/data-salary.txt")

In [3]:
stanmodel='''
data {
    int N;
    real X[N];
    real Y[N];
}

parameters {
    real a;
    real b;
    real<lower=0> sigma;
} 

model {
    for (n in 1:N){
    Y[n] ~ normal(a + b*X[n], sigma);
    }
}
'''

In [4]:
stan_input = { 'N':df.X.size, 'X':df.X, 'Y':df.Y}

In [5]:
sm = pystan.StanModel(model_code=stanmodel)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c8e14ada553109ca846f142bb100137d NOW.


CompileError: command 'gcc' failed with exit status 1